In [1]:
from htmd import *
from htmd.molecule.util import maxDistance
from htmd.builder.charmm import defaultParam, defaultTopo

htmd.config(viewer='ngl')


Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New devel HTMD version (1.7.34 python[==3.6,==3.5]) is available. You are currently on (1.7.33). Use 'conda update -c acellera htmd' to update to the new version. You might need to update your python version as well if there is no release for your current version.



In [2]:
# dockAllCenters runs docking of a given protein and ligand for all provided centers
def dockAllCenters(prot, ligand, centers):
    poses = []
    scores = []
    
    for i, center in enumerate(centers):
        currentCenterPoses, currentCenterScoring = dock(prot, ligand, center=center, extent=[ 15., 15., 15. ])
        poses.append(currentCenterPoses)
        scores.append(currentCenterScoring)
        
        print("Finished docking " + str(i+1) + "/" + str(len(centers)))
        
    return poses, scores

In [3]:
def buildSystemWithPose(prot, pose, paramFolder, ligandName="MOL"):
    # prepare protein
    prot = proteinPrepare(prot)
    prot.set('segid', 'P', sel='protein')
    prot.set('segid', 'W', sel='water')
    
    # prepare ligand (pose)
    pose.set('segid','L')
    pose.set('resname', ligandName) 
    
    mol = Molecule(name='combo')
    mol.append(prot)
    mol.append(pose)
    mol.center()
    
    # distance + extra
    D = maxDistance(mol, 'all') + 10
    
    # solvate
    smol = solvate(mol, minmax=[[-D, -D, -D], [D, D, D]])
    
    # build system    
    param = defaultParam()
    param.append(paramFolder + "mol.prm")

    topo = defaultTopo()
    topo.append(paramFolder + "mol.rtf")
    
    bmol = charmm.build(smol, param=param, topo=topo)
    
    return bmol

In [4]:
# load protein in pdb format
prot = Molecule('project_data/FXR_APO_structure_D3R_GC2.pdb')
prot.view()

In [5]:
# load all ligands in mol2 format
ligands = []
ligands.append(Molecule("project_data/ligand_FXR_1.sdf.mol2"))
ligands[0].view()

In [6]:
# binding pocket centers extracted with DeepSite (in playmolecule)
centers = [[43.9, 9.9, 3.2]  ]#,
           #[29.9, 7.9, 7.2],
           #[25.9, 3.9, -2.8],
           #[21.9, 3.9, -12.8],
           #[29.9, 19.9, -0.8]]

In [7]:
saveAllProtLigandToFile = True
saveBestPosesToFile = True

# write csv header if saving to file
scoresFile = None
if saveBestPosesToFile:
    scoresFile = open('ligand_scores.csv','w+')
    scoresFile.write("kcal1,rmsd_lb1,rmsd_ub1,kcal2,rmsd_lb2,rmsd_ub2\n")

for l, ligand in enumerate(ligands):
    bestRmsdPose, bestKcalPose = None, None
    bestRmsdScores, bestKcalScores = [float("inf")]*3, [float("inf")]*3
    
    poses, scores = dockAllCenters(prot, ligand, centers)
    for i, poseListPerCenter in enumerate(poses):
        for j, pose in enumerate(poseListPerCenter):
            if saveAllProtLigandToFile:
                mol = Molecule()
                mol.append(prot)
                mol.append(pose)
                mol.write("ligand_{}_center_{}_pose_{}.pdb".format(l+1, i+1, j+1))
                #mol.write("ligand_" + str(l+1) + "_center_" + str(i+1) + "_pose_" + str(j+1) + ".pdb")
                
            if scores[i][j][0] < bestKcalScores[0]:
                bestKcalScores = scores[i][j]
                bestKcalPose = pose
            
            if scores[i][j][2] < bestRmsdScores[2]:
                bestRmsdScores = scores[i][j]
                bestRmsdPose = pose
                
    

    if saveBestPosesToFile:
        # write pdb files
        bestKcalPose.write("selected_kcal_ligand_{}.mol2".format(l+1))
        bestRmsdPose.write("selected_rmsd_ligand_{}.mol2".format(l+1))
        #bestKcalPose.write("selected_kcal_ligand_" + str(l+1) + ".pdb")
        #bestRmsdPose.write("selected_rmsd_ligand_" + str(l+1) + ".pdb")
        
        # and csv with scores
        # format:
        # kcal1, rmsd_lb1, rmsd_ub1, kcal2, rmsd_lb2, rmsd_ub2
        # where 1 is the pose selected by best kcal and 2 is the pose selected by best rmsd        
        allScores = []
        allScores.extend(bestKcalScores)
        allScores.extend(bestRmsdScores)

        scoresFile.write(','.join(map(str, allScores))+"\n")    
        
    #bmol = buildSystemWithPose(prot, bestKcalPose, "project_data/param-ligand_FXR_1.sdf/parameters/CGenFF_2b6/b3lyp-cc-pVDZ-water/")
    #bmol.view(viewer='vmd')
    
    #tmpMol = bmol
    #tmpBestPose = bestKcalPose
    
if saveBestPosesToFile:
    scoresFile.close()

Finished docking 1/1


# Zona de testing

In [21]:
#tmpMol.view()
[1,2,3] + [4,5,6]

[1, 2, 3, 4, 5, 6]

## Build system

In [8]:
prot1 = Molecule("project_data/FXR_APO_structure_D3R_GC2.pdb")

prot1 = proteinPrepare(prot1)
prot1.set('segid', 'P', sel='protein')
prot1.set('segid', 'W', sel='water')


2017-06-16 01:01:55,177 - propka - INFO - No pdbfile provided
2017-06-16 01:02:06,143 - htmd.builder.preparationdata - INFO - The following residues are in a non-standard state: HIS   258  A (HID), HIS   298  A (HID), HIS   317  A (HIP), HIS   348  A (HID), HIS   426  A (HID), HIS   433  A (HIE), HIS   449  A (HID), HIS   450  A (HIE), HIS   451  A (HID), HIS   746  B (HID)
2017-06-16 01:02:06,144 - htmd.builder.preparationdata - WARNING - Dubious protonation state: the pKa of 8 residues is within 1.0 units of pH 7.0.
2017-06-16 01:02:06,146 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS   258  A (pKa= 6.38)
2017-06-16 01:02:06,147 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS   317  A (pKa= 7.61)
2017-06-16 01:02:06,147 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS   348  A (pKa= 6.08)
2017-06-16 01:02:06,148 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS   426  A 

In [9]:
prot1.view()

In [ ]:
#D = maxDistance(prot1, 'all')
#D

In [10]:
ligand1 = Molecule("selected_kcal_ligand_1.mol2")
ligand1.set('segid','L')
ligand1.set('resname', "MOL")

In [11]:
ligand1.view()

In [12]:
mol = Molecule(name='combo')
mol.append(prot1)
mol.append(ligand1)
mol.center()
mol.view()

In [ ]:
D = maxDistance(mol, 'all')
D

In [ ]:
D += 5
smol = solvate(mol, minmax=[[-D, -D, -D], [D, D, D]])

In [ ]:
smol.view(viewer='vmd')

In [ ]:
paramFolder = "param-ligand_FXR_1.sdf/parameters/CGenFF_2b6/b3lyp-cc-pVDZ-water/"

param = defaultParam()
param.append(paramFolder + "mol.prm")

topo = defaultTopo()
topo.append(paramFolder + "mol.rtf")

In [ ]:
bmol = charmm.build(smol, param=param, topo=topo)